# LangChainとLangGraphによる RAG・AIエージェント
3章：プロンプトエンジニアリング

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "xxx"

In [8]:
from openai import OpenAI

client = OpenAI()

## プロンプトのテンプレート化

In [5]:
prompt = """\
ユーザーが入力した料理のレシピを考えてください。

料理名: '''
{dish}
'''
"""

In [6]:
dish = "カレー"
prompt.format(dish=dish)

"ユーザーが入力した料理のレシピを考えてください。\n\n料理名: '''\nカレー\n'''\n"

In [9]:
def generate_recipe(dish: str) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "ユーザーが入力した料理のレシピを考えてください。"},
            {"role": "user", "content": f"{dish}"},
        ],
    )
    return response.choices[0].message.content


recipe = generate_recipe("カレー")
print(recipe)

もちろん、カレーのレシピを提案します。基本的なチキンカレーのレシピで、あなたのお好みでアレンジも可能です。

### 材料 (4人分)

- 鶏肉（もも肉または胸肉） 400g
- 玉ねぎ 2個
- にんにく 2かけ
- 生姜 1かけ
- トマト 1個（またはホールトマト缶 1缶）
- カレーパウダー 大さじ2～3（お好みで調整）
- クミンシード 小さじ1
- ココナッツミルク 200ml（お好みで）
- サラダ油 大さじ2
- 塩 適量
- 胡椒 適量
- 水 300ml
- パクチー（飾り用、オプション）

### 作り方

1. **下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒を振って下味をつけておきます。
   - 玉ねぎ、にんにく、生姜はそれぞれみじん切りにします。
   - トマトは角切りにします（缶詰を使用する場合はそのまま使います）。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、クミンシードを加えて香りが立つまで炒めます。
   - 次に、みじん切りにした玉ねぎを加えて透明になるまで炒めます。

3. **香辛料の追加**:
   - にんにくと生姜を加え、さらに1～2分炒めます。
   - カレーパウダーを加え、全体に香辛料の香りが立つまでしっかり混ぜて炒めます。

4. **鶏肉を加える**:
   - 鶏肉を加え、表面が白くなるまで炒めます。

5. **煮込む**:
   - トマトと水を加え、沸騰させたら火を弱めて蓋をし、15～20分煮込みます。
   - ココナッツミルクを加え、さらに5分ほど煮込みます。お好みで塩と胡椒で味を調えてください。

6. **盛り付け**:
   - 皿にご飯を盛り、その上にカレーをかけます。お好みでパクチーを飾って完成です。

### サーブのアイデア
- ナンやライスと一緒にサーブすると美味しいです。
- サラダやラッシーと合わせると、バランスの良い食事になります。

楽しんでお料理してください！


### 出力方法を指定

In [11]:
system_prompt = """\
ユーザーが入力した料理のレシピを考えてください。

出力は以下のJSON形式にしてください。

```
{
  "材料": ["材料1", "材料2"],
  "手順": ["手順1", "手順2"]
}
```
"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": "カレー"},
    ],
)
print(response.choices[0].message.content)


```json
{
  "材料": ["玉ねぎ", "にんじん", "じゃがいも", "鶏肉", "カレールー", "水", "サラダ油", "塩"],
  "手順": [
    "玉ねぎ、にんじん、じゃがいもをそれぞれいちょう切りにする。",
    "フライパンにサラダ油を熱し、玉ねぎをしんなりするまで炒める。",
    "鶏肉を加えて表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、軽く炒める。",
    "水を加えて沸騰させ、アクを取る。",
    "弱火にして、野菜が柔らかくなるまで煮る。",
    "カレールーを加えてよく溶かし、さらに10分ほど煮込む。",
    "塩で味を調整して完成。"
  ]
}
```


## Zero-shot プロンプティング

In [12]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": "入力をポジティブ・ネガティブ・中立のどれかに分類してください。",
        },
        {
            "role": "user",
            "content": "ChatGPTはとても便利だ",
        },
    ],
)
print(response.choices[0].message.content)

ポジティブ


## Few-shot プロンプティング

In [13]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},
        {"role": "user", "content": "ChatGPTはとても便利だ"},
    ],
)
print(response.choices[0].message.content)

はい、ChatGPTは多くの用途に役立つ便利なツールです。情報提供、質問への回答、アイデアの提案、会話のサポートなど、さまざまな目的で利用されています。何か具体的にお手伝いできることがありますか？


In [14]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "入力がAIに関係するか回答してください。"},
        {"role": "user", "content": "AIの進化はすごい"},
        {"role": "assistant", "content": "true"},
        {"role": "user", "content": "今日は良い天気だ"},
        {"role": "assistant", "content": "false"},
        {"role": "user", "content": "ChatGPTはとても便利だ"},
    ],
)
print(response.choices[0].message.content)

true


## Zero-shot Chain-of-Thought プロンプティング

In [15]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "回答だけ一言で出力してください。"},
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},
    ],
)
print(response.choices[0].message.content)

10


In [16]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "ステップバイステップで考えてください。"},
        {"role": "user", "content": "10 + 2 * 3 - 4 * 2"},
    ],
)
print(response.choices[0].message.content)

計算をステップバイステップで行います。

1. まず、乗算を先に計算します。
   - \(2 * 3 = 6\)
   - \(4 * 2 = 8\)

2. これらをもとの式に戻します。
   - \(10 + 6 - 8\)

3. 次に、左から右へと加算と減算を行います。
   - \(10 + 6 = 16\)
   - \(16 - 8 = 8\)

したがって、最終的な答えは \(8\) です。
